# 0. Preparation

In [1]:
import sys
sys.path.append('../dataset/')
sys.path.append('../network/')
sys.path.append('../model/')

import os
import glob
import time
import torch
import argparse
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision.transforms as transforms

from PIL import Image
from pathlib import Path
from main_loading import *
from main_network import *
from main_model_one_class import *
from main_model_rec import *

In [2]:
data_tshirt = load_dataset(loader_name='fmnist_eval',
                           label_eval=(0,))

data_trouser = load_dataset(loader_name='fmnist_eval',
                            label_eval=(1,))

data_pullover = load_dataset(loader_name='fmnist_eval',
                             label_eval=(2,))

data_dress = load_dataset(loader_name='fmnist_eval',
                          label_eval=(3,))

data_coat = load_dataset(loader_name='fmnist_eval',
                         label_eval=(4,))

data_sandal = load_dataset(loader_name='fmnist_eval',
                           label_eval=(5,))

data_shirt = load_dataset(loader_name='fmnist_eval',
                          label_eval=(6,))

data_sneaker = load_dataset(loader_name='fmnist_eval',
                            label_eval=(7,))

data_bag = load_dataset(loader_name='fmnist_eval',
                        label_eval=(8,))

data_boot = load_dataset(loader_name='fmnist_eval',
                         label_eval=(9,))

# 1. Metric 1: $L^{2}$-Distance for Mean of Raw Images

## 1.0. Calculating Mean Vector For Each Dataset

In [3]:
dataset_dict = {'tshirt':data_tshirt, 
                'trouser': data_trouser, 
                'pullover': data_pullover, 
                'dress': data_dress, 
                'coat': data_coat,
                'sandal': data_sandal, 
                'shirt': data_shirt, 
                'sneaker': data_sneaker, 
                'bag': data_bag, 
                'boot': data_boot}

In [4]:
batch_size_loading = 7000
num_workers = 6
mean_dict = {}
untransform = transforms.ToPILImage()

In [5]:
for key, dataset in sorted(dataset_dict.items()):
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        break
    
    X_mean = torch.mean(X, axis=0)
    mean_dict[key] = X_mean
    img = untransform(X_mean); print(key); display(img)

bag


boot


coat


dress


pullover


sandal


shirt


sneaker


trouser


tshirt


## 1.1. Obtaining $L^2$ Distance

### 1.1.1. Get `l2_distance_df`

In [6]:
l2_distance_matrix = np.zeros((10, 10))
keys = sorted(dataset_dict.keys())
l2_binary_dict = dict()
criterion = nn.MSELoss(reduction='none')

In [7]:
for i, key_i in enumerate(keys):
    for j, key_j in enumerate(keys):
        x_i, x_j = mean_dict[key_i], mean_dict[key_j]
        dist = criterion(x_i, x_j)
        dist_mean = round(torch.mean(dist).cpu().data.numpy().item(), 3)
        l2_distance_matrix[i][j] = dist_mean

In [8]:
l2_distance_df = pd.DataFrame(l2_distance_matrix)
l2_distance_df.index = keys
l2_distance_df.columns = keys
# l2_distance_df.sort_values('coat')
# l2_distance_df
l2_distance_df

,bag,boot,coat,dress,pullover,sandal,shirt,sneaker,trouser,tshirt
bag,0.000,0.048,0.047,0.085,0.042,0.072,0.036,0.070,0.120,0.073
boot,0.048,0.000,0.106,0.121,0.095,0.063,0.085,0.066,0.147,0.117
coat,0.047,0.106,0.000,0.060,0.008,0.136,0.012,0.142,0.089,0.043
dress,0.085,0.121,0.060,0.000,0.065,0.100,0.035,0.116,0.021,0.026
pullover,0.042,0.095,0.008,0.065,0.000,0.116,0.009,0.127,0.091,0.040
sandal,0.072,0.063,0.136,0.100,0.116,0.000,0.088,0.019,0.105,0.115
shirt,0.036,0.085,0.012,0.035,0.009,0.088,0.000,0.103,0.062,0.017
sneaker,0.070,0.066,0.142,0.116,0.127,0.019,0.103,0.000,0.125,0.134
trouser,0.120,0.147,0.089,0.021,0.091,0.105,0.062,0.125,0.000,0.053
tshirt,0.073,0.117,0.043,0.026,0.040,0.115,0.017,0.134,0.053,0.000


In [9]:
l2_distance_df['sandal'].sort_values()

sandal      0.000
sneaker     0.019
boot        0.063
bag         0.072
shirt       0.088
dress       0.100
trouser     0.105
tshirt      0.115
pullover    0.116
coat        0.136
Name: sandal, dtype: float64

In [9]:
l2_distance_df['coat'].sort_values()

coat        0.000
pullover    0.008
shirt       0.012
tshirt      0.043
bag         0.047
dress       0.060
trouser     0.089
boot        0.106
sandal      0.136
sneaker     0.142
Name: coat, dtype: float64

In [10]:
l2_distance_df['pullover'].sort_values()

pullover    0.000
coat        0.008
shirt       0.009
tshirt      0.040
bag         0.042
dress       0.065
trouser     0.091
boot        0.095
sandal      0.116
sneaker     0.127
Name: pullover, dtype: float64

In [11]:
l2_distance_df['shirt'].sort_values()

shirt       0.000
pullover    0.009
coat        0.012
tshirt      0.017
dress       0.035
bag         0.036
trouser     0.062
boot        0.085
sandal      0.088
sneaker     0.103
Name: shirt, dtype: float64

In [12]:
l2_distance_df['tshirt'].sort_values()

tshirt      0.000
shirt       0.017
dress       0.026
pullover    0.040
coat        0.043
trouser     0.053
bag         0.073
sandal      0.115
boot        0.117
sneaker     0.134
Name: tshirt, dtype: float64

> The **maximum** distance is in `<boot, trouser>`, which is **0.147**.   
> The **minimum** distance is in `<coat, pullover>`, which is **0.008**.

> The ranking for `coat` is: 
- **`pullover`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `dress` < `bag` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `tshirt` is: 
- **`shirt`** < `dress` < `pullover` < `coat` < `trouser` < `bag` < `sandal` < `boot` < **`sneaker`**  

### 1.1.2. Get `l2_binary_df`

In [13]:
for i, key in enumerate(keys):
    distances = l2_distance_df[key].values.copy()
    max_ = keys[np.argmax(distances)]
    distances[i] = np.inf
    min_ = keys[np.argmin(distances)]
    l2_binary_dict[key] = dict()
    l2_binary_dict[key]['min'] = min_
    l2_binary_dict[key]['max'] = max_

In [14]:
l2_binary_df = pd.DataFrame(l2_binary_dict).T
l2_binary_df

,max,min
bag,trouser,shirt
boot,trouser,bag
coat,sneaker,pullover
dress,boot,trouser
pullover,sneaker,coat
sandal,coat,sneaker
shirt,sneaker,pullover
sneaker,coat,sandal
trouser,boot,dress
tshirt,sneaker,shirt


# 2. Metric 2: $L^{1}$-Distance for Mean of Raw Images

## 2.1. Obtaining $L^1$ Distance

### 2.1.1. Get `l1_distance_df`

In [15]:
l1_distance_matrix = np.zeros((10, 10))
l1_binary_dict = dict()

In [16]:
for i, key_i in enumerate(keys):
    for j, key_j in enumerate(keys):
        x_i, x_j = mean_dict[key_i], mean_dict[key_j]
        dist = torch.abs(x_i - x_j)
        dist_mean = round(torch.mean(dist).cpu().data.numpy().item(), 3)
        l1_distance_matrix[i][j] = dist_mean

In [17]:
l1_distance_df = pd.DataFrame(l1_distance_matrix)
l1_distance_df.index = keys
l1_distance_df.columns = keys
l1_distance_df

,bag,boot,coat,dress,pullover,sandal,shirt,sneaker,trouser,tshirt
bag,0.000,0.165,0.152,0.231,0.151,0.218,0.149,0.205,0.278,0.208
boot,0.165,0.000,0.232,0.258,0.229,0.182,0.226,0.183,0.283,0.265
coat,0.152,0.232,0.000,0.159,0.066,0.297,0.088,0.284,0.197,0.140
dress,0.231,0.258,0.159,0.000,0.177,0.240,0.142,0.241,0.082,0.103
pullover,0.151,0.229,0.066,0.177,0.000,0.278,0.066,0.273,0.217,0.114
sandal,0.218,0.182,0.297,0.240,0.278,0.000,0.237,0.089,0.235,0.275
shirt,0.149,0.226,0.088,0.142,0.066,0.237,0.000,0.246,0.191,0.088
sneaker,0.205,0.183,0.284,0.241,0.273,0.089,0.246,0.000,0.241,0.280
trouser,0.278,0.283,0.197,0.082,0.217,0.235,0.191,0.241,0.000,0.158
tshirt,0.208,0.265,0.140,0.103,0.114,0.275,0.088,0.280,0.158,0.000


In [18]:
l1_distance_df['coat'].sort_values()

coat        0.000
pullover    0.066
shirt       0.088
tshirt      0.140
bag         0.152
dress       0.159
trouser     0.197
boot        0.232
sneaker     0.284
sandal      0.297
Name: coat, dtype: float64

In [19]:
l1_distance_df['pullover'].sort_values()

pullover    0.000
coat        0.066
shirt       0.066
tshirt      0.114
bag         0.151
dress       0.177
trouser     0.217
boot        0.229
sneaker     0.273
sandal      0.278
Name: pullover, dtype: float64

In [20]:
l1_distance_df['shirt'].sort_values()

shirt       0.000
pullover    0.066
coat        0.088
tshirt      0.088
dress       0.142
bag         0.149
trouser     0.191
boot        0.226
sandal      0.237
sneaker     0.246
Name: shirt, dtype: float64

In [21]:
l1_distance_df['tshirt'].sort_values()

tshirt      0.000
shirt       0.088
dress       0.103
pullover    0.114
coat        0.140
trouser     0.158
bag         0.208
boot        0.265
sandal      0.275
sneaker     0.280
Name: tshirt, dtype: float64

> The **maximum** distance is in `<coat, sandal>`, which is **0.297**.   
> The **minimum** distance is in `<coat, pullover>`, which is **0.066**.

> The ranking for `coat` is: 
- **`pullover`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sneaker` < **`sandal`**

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `dress` < `bag` < `trouser` < `boot` < `sandal` < **`sneaker`**  

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `bag` < `dress` < `trouser` < `boot` < `sneaker` < **`sandal`**  

> The ranking for `tshirt` is: 
- **`shirt`** < `dress` < `pullover` < `coat` < `trouser` < `bag` < `boot` < `sandal` < **`sneaker`**  

### 2.1.2. Get `l1_binary_df`

In [22]:
for i, key in enumerate(keys):
    distances = l1_distance_df[key].values.copy()
    max_ = keys[np.argmax(distances)]
    distances[i] = np.inf
    min_ = keys[np.argmin(distances)]
    l1_binary_dict[key] = dict()
    l1_binary_dict[key]['min'] = min_
    l1_binary_dict[key]['max'] = max_

In [23]:
l1_binary_df = pd.DataFrame(l1_binary_dict).T

In [24]:
(l2_binary_df == l1_binary_df).sum()

max     8
min    10
dtype: int64

> The results are similar to that of $L^2$-distance's.

# 3. Metric 3: $L^{2}$-Distance for Mean of Representations in Reconstruction Model (Unsupervised)

> **Step 1**: We will use coat-train to train an unsupervised reconstruction model.  
> **Step 2**: We will use coat-test to calculate the mean of representations (outputs).   
> **Step 3**: We will use other other datasets to calculate the mean of representations.  
> **Step 4**: We compare the distances between these means.   

## Shirt

In [44]:
label_eval=(6,)
base = 'shirt'

In [45]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                          num_workers=num_workers)


for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)
    
X_pred_mean_base = torch.mean(X_pred, axis=0)


for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

In [46]:
rec_distance_df_shirt = pd.DataFrame(rec_distance_dict)
rec_distance_df_shirt.sort_values(base, inplace=True)
rec_distance_df_shirt

,shirt
shirt,0.000000
pullover,0.000620
coat,0.000726
tshirt,0.001979
bag,0.002640
dress,0.002952
boot,0.003957
trouser,0.004746
sandal,0.004897
sneaker,0.005144


In [47]:
rec_ranking_shirt = rec_distance_df_shirt.index
rec_ranking_shirt

Index(['shirt', 'pullover', 'coat', 'tshirt', 'bag', 'dress', 'boot',
       'trouser', 'sandal', 'sneaker'],
      dtype='object')

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `bag` < `dress` < `boot` < `trouser` < `sandal` < **`sneaker`**  

## Pullover

In [48]:
label_eval=(2,)
base = 'pullover'

In [49]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                          num_workers=num_workers)


for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)
    
X_pred_mean_base = torch.mean(X_pred, axis=0)


for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

In [50]:
rec_distance_df_pullover = pd.DataFrame(rec_distance_dict)
rec_distance_df_pullover.sort_values(base, inplace=True)
rec_distance_df_pullover

,pullover
pullover,0.000000
coat,0.000449
shirt,0.000606
tshirt,0.000968
dress,0.001964
trouser,0.002877
bag,0.003092
boot,0.005315
sneaker,0.006766
sandal,0.008275


In [51]:
rec_ranking_pullover = rec_distance_df_pullover.index
rec_ranking_pullover

Index(['pullover', 'coat', 'shirt', 'tshirt', 'dress', 'trouser', 'bag',
       'boot', 'sneaker', 'sandal'],
      dtype='object')

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

## tshirt

In [52]:
label_eval=(0,)
base = 'tshirt'

In [53]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                          num_workers=num_workers)


for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)
    
X_pred_mean_base = torch.mean(X_pred, axis=0)


for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

In [54]:
rec_distance_df_tshirt = pd.DataFrame(rec_distance_dict)
rec_distance_df_tshirt.sort_values(base, inplace=True)
rec_distance_df_tshirt

,tshirt
tshirt,0.000000
pullover,0.000860
shirt,0.001122
coat,0.001289
dress,0.002390
trouser,0.004361
bag,0.005570
boot,0.008033
sneaker,0.008674
sandal,0.012165


In [55]:
rec_ranking_tshirt = rec_distance_df_tshirt.index
rec_ranking_tshirt

Index(['tshirt', 'pullover', 'shirt', 'coat', 'dress', 'trouser', 'bag',
       'boot', 'sneaker', 'sandal'],
      dtype='object')

> The ranking for `tshirt` is: 
- **`pullover`** < `shirt` < `coat` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

## 3.1. Get Mean

In [33]:
label_eval=(4,)
base = 'coat'

In [31]:
rec_distance_dict = {base:{base: 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_rec'
model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[{}]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'.format(label_eval[0])

In [34]:
net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

<All keys matched successfully>

In [36]:
data_base_test = load_dataset(loader_name='fmnist_eval',
                              label_eval=label_eval,
                              test_eval=True)

base_loader_test = data_base_test.loaders(batch_size=batch_size_loading,
                                     num_workers=num_workers)

In [38]:
for data in base_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)

In [39]:
X_pred_mean_base = torch.mean(X_pred, axis=0)

In [40]:
display(untransform(X_pred_mean_base))

## 3.2. Get Distances

In [41]:
for key, dataset in sorted(dataset_dict.items()):
    if key == base:
        continue
    print(key)
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_base, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    rec_distance_dict[base][key] = dist_mean

bag
boot
dress
pullover
sandal
shirt
sneaker
trouser
tshirt


In [42]:
rec_distance_df = pd.DataFrame(rec_distance_dict)
rec_distance_df.sort_values(base, inplace=True)
rec_distance_df

,coat
coat,0.000000
pullover,0.000900
shirt,0.001395
bag,0.002755
tshirt,0.003577
boot,0.006922
dress,0.007182
sneaker,0.009563
sandal,0.010589
trouser,0.011027


In [43]:
rec_ranking = rec_distance_df.index
rec_ranking

Index(['coat', 'pullover', 'shirt', 'bag', 'tshirt', 'boot', 'dress',
       'sneaker', 'sandal', 'trouser'],
      dtype='object')

> The **maximum** distance is in `<coat, sandal>`, which is **0.011**.   
> The **minimum** distance is in `<coat, pullover>`, which is **0.001**.  

> The ranking for `coat` is: 
- **`pullover`** < `shirt` < `bag` < `tshirt` < `boot` < `dress` < `sneaker` < `sandal` < **`trouser`**  

> The ranking for `shirt` is: 
- **`pullover`** < `coat` < `tshirt` < `bag` < `dress` < `boot` < `trouser` < `sandal` < **`sneaker`**   

> The ranking for `pullover` is: 
- **`coat`** < `shirt` < `tshirt` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

> The ranking for `tshirt` is: 
- **`pullover`** < `shirt` < `coat` < `dress` < `trouser` < `bag` < `boot` < `sneaker` < **`sandal`**  

# 4. Metric 4: $L^{2}$-Distance for Mean of Representations in One-Class Model (Unsupervised, w/o Pretraining)

> **Step 1**: We will use coat-train to train an unsupervised one-class model.  
> **Step 2**: We will use coat-test to calculate the mean of representations (outputs).   
> **Step 3**: We will use other other datasets to calculate the mean of representations.  
> **Step 4**: We compare the distances between these means.   

## 4.1. Get Mean

In [361]:
oc_distance_dict = {'coat':{'coat': 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_one_class'
model_path = '/net/leksai/nips/model/one_class/fmnist/one_class_unsupervised_[4]_[]_[0]_[0.0]/net_fmnist_LeNet_one_class_eta_100_epochs_150_batch_128/model.tar'

In [362]:
net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

<All keys matched successfully>

In [363]:
for data in coat_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)

In [364]:
X_pred_mean_coat = torch.mean(X_pred, axis=0)

## 4.2. Get Distances

In [365]:
for key, dataset in sorted(dataset_dict.items()):
    if key == 'coat':
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_coat, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    oc_distance_dict['coat'][key] = dist_mean

In [378]:
oc_distance_df = pd.DataFrame(oc_distance_dict)
oc_distance_df.sort_values('coat', inplace=True)
oc_distance_df

,coat
coat,0.000000
sneaker,0.000007
bag,0.000008
tshirt,0.000013
dress,0.000014
boot,0.000018
sandal,0.000032
pullover,0.000044
shirt,0.000069
trouser,0.000212


In [381]:
oc_ranking = oc_distance_df.index
oc_ranking

Index(['coat', 'sneaker', 'bag', 'tshirt', 'dress', 'boot', 'sandal',
       'pullover', 'shirt', 'trouser'],
      dtype='object')

> The **maximum** distance is in `<coat, trouser>`, which is **0.000212**.   
> The **minimum** distance is in `<coat, sneaker>`, which is **0.000007**.  

> The ranking for `coat` is: 
- **`sneaker`** < `bag` < `tshirt` < `dress` < `boot` < `sandal` < `pullover` < `shirt` < **`trouser`**

> The ranking is quite different to raw images, and the magnitudes are smaller.

# 5. Metric 5: $L^{2}$-Distance for Mean of Representations in One-Class Model (Unsupervised, w/ Pretraining)

> **Step 1**: We will use coat-train to train an unsupervised one-class model.  
> **Step 2**: We will use coat-test to calculate the mean of representations (outputs).   
> **Step 3**: We will use other other datasets to calculate the mean of representations.  
> **Step 4**: We compare the distances between these means.   

## 5.1. Get Mean

In [353]:
oc_pt_distance_dict = {'coat':{'coat': 0.000}}
device = 'cpu'
net_name = 'fmnist_LeNet_one_class'
model_path = '/net/leksai/nips/model/one_class/fmnist/one_class_unsupervised_[4]_[]_[1]_[0.0]/net_fmnist_LeNet_one_class_eta_100_epochs_150_batch_128/model.tar'

In [354]:
net = build_network(net_name)
model_dict = torch.load(model_path, map_location=device)
net.load_state_dict(model_dict['net_dict'])

<All keys matched successfully>

In [355]:
for data in coat_loader_test:
    X, y, idx = data
    X, y, idx = X.to(device), y.to(device), idx.to(device)
    X_pred = net(X)

In [356]:
X_pred_mean_coat = torch.mean(X_pred, axis=0)

## 5.2. Get Distances

In [411]:
for key, dataset in sorted(dataset_dict.items()):
    if key == 'coat':
        continue
    loader = dataset.loaders(batch_size=batch_size_loading,
                             num_workers=num_workers)
    for data in loader:
        X, y, idx = data
        X, y, idx = X.to(device), y.to(device), idx.to(device)
        break
    
    X_pred = net(X)
    X_pred_mean = torch.mean(X_pred, axis=0)
    dist = criterion(X_pred_mean_coat, X_pred_mean)
    dist_mean = torch.mean(dist).cpu().data.numpy().item()
    oc_pt_distance_dict['coat'][key] = dist_mean

In [371]:
oc_pt_distance_df = pd.DataFrame(oc_pt_distance_dict)
oc_pt_distance_df.sort_values('coat', inplace=True)

In [372]:
oc_pt_distance_df

,coat
coat,0.000000
bag,0.000007
sneaker,0.000007
dress,0.000009
tshirt,0.000012
boot,0.000013
sandal,0.000028
pullover,0.000049
shirt,0.000086
trouser,0.000220


In [380]:
oc_pt_ranking = oc_pt_distance_df.index
oc_pt_ranking

Index(['coat', 'bag', 'sneaker', 'dress', 'tshirt', 'boot', 'sandal',
       'pullover', 'shirt', 'trouser'],
      dtype='object')

> The **maximum** distance is in `<coat, trouser>`, which is **0.000200**.   
> The **minimum** distance is in `<coat, bag>` and `<coat, sneaker>`, which are respectively **6.6e-6** and **7.7e-6**.  

> The ranking for `coat` is: 
- **`bag`** < `sneaker` < `dress` < `tshirt` < `boot` < `sandal` < `pullover` < `shirt` < **`trouser`** 

> The ranking is quite different to raw images, and the magnitudes are smaller.